In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from tqdm import tqdm

from datetime import datetime

In [2]:
from functions import get_energy


In [3]:
df=get_energy.get()

  0%|          | 0/256 [00:00<?, ?it/s]/Users/stephantimpe/PycharmProjects/probabilistic-forecasting-challenge/functions/get_energy.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  energydata = pd.concat([energydata, pd.DataFrame(rawdata, columns=col_names)])
100%|██████████| 256/256 [00:29<00:00,  8.62it/s]


In [4]:
df.head()

,gesamt,weekday
date_time,,
2018-12-24 00:00:00,42.02925,0
2018-12-24 01:00:00,39.61025,0
2018-12-24 02:00:00,39.13875,0
2018-12-24 03:00:00,39.42100,0
2018-12-24 04:00:00,40.74775,0


Define weekday column

In [5]:

#df["time"] = df.index.strftime("%H:%M")

Lead times are

In [6]:
horizons_def = [36, 40, 44, 60, 64, 68]#[24 + 12*i for i in range(5)]
horizons_def

[36, 40, 44, 60, 64, 68]

Adapt horzions so they actually fit

In [7]:
horizons = [h+1 for h in horizons_def]
horizons

[37, 41, 45, 61, 65, 69]

In [8]:
def get_date_from_horizon(last_ts, horizon):
    return last_ts + pd.DateOffset(hours=horizon)

In [9]:
LAST_IDX = -1
LAST_DATE = df.iloc[LAST_IDX].name
LAST_DATE

Timestamp('2023-11-19 10:00:00')

In [10]:
from datetime import datetime, timedelta

# Get the current date
current_date = datetime.now()

# Calculate the number of days to add or subtract to get to Thursday
# In Python's datetime module, Monday is 0 and Sunday is 6
days_until_thursday = 3 - current_date.weekday()  # 3 represents Thursday

# If it's already past Thursday, calculate for the next Thursday
if days_until_thursday < 0:
    days_until_thursday += 7

# Get the Thursday of the current week
thursday_of_current_week = current_date + timedelta(days=days_until_thursday)

# To set the time to 00:00, replace the hour, minute, second, and microsecond
thursday_of_current_week = thursday_of_current_week.replace(hour=0, minute=0, second=0, microsecond=0)

print("Thursday of the current week:", thursday_of_current_week)
LAST_DATE=thursday_of_current_week


Thursday of the current week: 2023-11-23 00:00:00


Get time and date that correspond to the lead times (starting at the last observation in our data which should be the respective thursday 0:00)  
*Attention*: if the last timestamp in the data is not thursday 0:00, you have to adjust your lead times accordingly

In [11]:
horizon_date = [get_date_from_horizon(LAST_DATE, h) for h in horizons]
horizon_date

[Timestamp('2023-11-24 13:00:00'),
 Timestamp('2023-11-24 17:00:00'),
 Timestamp('2023-11-24 21:00:00'),
 Timestamp('2023-11-25 13:00:00'),
 Timestamp('2023-11-25 17:00:00'),
 Timestamp('2023-11-25 21:00:00')]

quantile levels

In [12]:
tau = [.025, .25, .5, .75, .975]

In [13]:
#rows correspond to horizon, columns to quantile level
pred_baseline = np.zeros((6,5))

Seasonal regression

In [14]:
#seasonal regression
# Create dummy variables for months and hours
df['month'] = df.index.month
df['hour'] = df.index.hour

# Get dummies for months and hours, excluding the first month and hour to avoid multicollinearity
month_dummies = pd.get_dummies(df['month'], prefix='month', drop_first=True)
hour_dummies = pd.get_dummies(df['hour'], prefix='hour', drop_first=True)

# Join the dummies with the original DataFrame
df = df.join(month_dummies).join(hour_dummies)


In [15]:
df
print(df.dtypes)

gesamt      float64
weekday       int32
month         int32
hour          int32
month_2        bool
month_3        bool
month_4        bool
month_5        bool
month_6        bool
month_7        bool
month_8        bool
month_9        bool
month_10       bool
month_11       bool
month_12       bool
hour_1         bool
hour_2         bool
hour_3         bool
hour_4         bool
hour_5         bool
hour_6         bool
hour_7         bool
hour_8         bool
hour_9         bool
hour_10        bool
hour_11        bool
hour_12        bool
hour_13        bool
hour_14        bool
hour_15        bool
hour_16        bool
hour_17        bool
hour_18        bool
hour_19        bool
hour_20        bool
hour_21        bool
hour_22        bool
hour_23        bool
dtype: object


In [16]:
#seasonal regression
import statsmodels.api as sm

# Define the independent variables (X) and the dependent variable (y)
# Exclude the original 'month' and 'hour' columns to avoid multicollinearity
exclude_columns = ['gesamt', 'month', 'hour']
X = df[[col for col in df.columns if col not in exclude_columns]]
y = df['gesamt']
X = X.apply(pd.to_numeric)
y = y.apply(pd.to_numeric)

for column in X.columns:
    X[column] = X[column].astype(int)


# Add a constant to the model (for the intercept)
X = sm.add_constant(X)

# Fit the model
model = sm.OLS(y, X).fit()

# Print out the statistics
print(model.summary())
'quantile reg'

                            OLS Regression Results                            
Dep. Variable:                 gesamt   R-squared:                       0.711
Model:                            OLS   Adj. R-squared:                  0.711
Method:                 Least Squares   F-statistic:                     3017.
Date:                Sun, 19 Nov 2023   Prob (F-statistic):               0.00
Time:                        11:29:12   Log-Likelihood:            -1.3237e+05
No. Observations:               42995   AIC:                         2.648e+05
Df Residuals:                   42959   BIC:                         2.651e+05
Df Model:                          35                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         57.3880      0.154    372.748      0.0

'quantile reg'

In [19]:
# Assuming horizon_dates is a list of pandas Timestamps
horizon_dates = [pd.Timestamp('2023-11-17 13:00:00'),
                 pd.Timestamp('2023-11-17 17:00:00'),
                 pd.Timestamp('2023-11-17 21:00:00'),
                 pd.Timestamp('2023-11-18 13:00:00'),
                 pd.Timestamp('2023-11-18 17:00:00'),
                 pd.Timestamp('2023-11-18 21:00:00')]

# Create a new DataFrame for the forecast
forecast_df = pd.DataFrame({'date_time': horizon_dates})
# Extract and one-hot encode month and hour from the date_time
forecast_df['month'] = forecast_df['date_time'].dt.month
forecast_df['hour'] = forecast_df['date_time'].dt.hour
forecast_df = forecast_df.join(pd.get_dummies(forecast_df['month'], prefix='month', drop_first=True))
forecast_df = forecast_df.join(pd.get_dummies(forecast_df['hour'], prefix='hour', drop_first=True))

# Drop the original month and hour columns
forecast_df.drop(['month', 'hour'], axis=1, inplace=True)

# Add constant and weekday columns
forecast_df['const'] = 1.0
forecast_df['weekday'] = forecast_df['date_time'].dt.dayofweek

# Ensure all columns in X are also in forecast_df
for col in X.columns:
    if col not in forecast_df.columns and col != 'date_time':
        forecast_df[col] = 0

# Reorder columns to match X (excluding date_time)
forecast_columns = [col for col in X.columns if col != 'date_time']
forecast_df = forecast_df[['date_time'] + forecast_columns]

# Now forecast_df is ready for prediction
# Drop the date_time column for prediction
forecast_df_for_prediction = forecast_df.drop('date_time', axis=1)

# Make predictions
predictions = model.predict(forecast_df_for_prediction)

# Combine predictions with date_times
forecast_df['prediction'] = predictions
print(forecast_df[['date_time', 'prediction']])
forecast_df


            date_time prediction
0 2023-11-17 13:00:00  50.287837
1 2023-11-17 17:00:00  63.822624
2 2023-11-17 21:00:00  58.875788
3 2023-11-18 13:00:00  48.512805
4 2023-11-18 17:00:00  62.047592
5 2023-11-18 21:00:00  57.100756


,date_time,const,weekday,month_2,month_3,month_4,month_5,month_6,month_7,month_8,...,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,prediction
0,2023-11-17 13:00:00,1.0,4,0,0,0,0,0,0,0,...,0,0,False,0,0,0,False,0,0,50.287837
1,2023-11-17 17:00:00,1.0,4,0,0,0,0,0,0,0,...,0,0,True,0,0,0,False,0,0,63.822624
2,2023-11-17 21:00:00,1.0,4,0,0,0,0,0,0,0,...,0,0,False,0,0,0,True,0,0,58.875788
3,2023-11-18 13:00:00,1.0,5,0,0,0,0,0,0,0,...,0,0,False,0,0,0,False,0,0,48.512805
4,2023-11-18 17:00:00,1.0,5,0,0,0,0,0,0,0,...,0,0,True,0,0,0,False,0,0,62.047592
5,2023-11-18 21:00:00,1.0,5,0,0,0,0,0,0,0,...,0,0,False,0,0,0,True,0,0,57.100756


In [ ]:
X.head()

In [ ]:
predictions

Visually check if quantiles make sense

In [ ]:
x = horizons
_ = plt.plot(x,pred_baseline, ls="", marker="o", c="black")
_ = plt.xticks(x, x)
_ = plt.plot((x,x),(pred_baseline[:,0], pred_baseline[:,-1]),c='black')

In [ ]:
from datetime import datetime, date, timedelta
date_str = datetime.today().strftime('%Y%m%d')

In [ ]:
date_str = date.today() #- timedelta(days=1)
date_str = date_str
date_str

In [ ]:
df_sub = pd.DataFrame({
    "forecast_date": date_str,
    "target": "energy",
    "horizon": [str(h) + " hour" for h in horizons_def],
    "q0.025": pred_baseline[:,0],
    "q0.25": pred_baseline[:,1],
    "q0.5": pred_baseline[:,2],
    "q0.75": pred_baseline[:,3],
    "q0.975": pred_baseline[:,4]})
df_sub

In [ ]:
#need to change this
PATH = "../forecasts/"
date_str = date_str.strftime('%Y%m%d')

df_sub.to_csv(PATH+date_str+"_power_benchmark.csv", index=False)